In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import torch
import torch.nn as nn

In [ ]:
#darknet + Yolo
config = [
    (32, 3, 1),  #(filter, size x size, repeat)
    (64, 3, 2),
    ["B", 1],    #one time residual block
    (128, 3, 2), #single convolutional layer
    ["B", 2],    #two time residual block...
    (256, 3, 2),
    ["B", 8],
    (512, 3, 2),
    ["B", 8],
    (1024, 3, 2),
    ["B", 4],  # To this point is Darknet-53
    (512, 1, 1),
    (1024, 3, 1),
    "S",        #82
    (256, 1, 1),
    "U",
    (256, 1, 1),
    (512, 3, 1),
    "S",        #94
    (128, 1, 1),
    "U",
    (128, 1, 1),
    (256, 3, 1),
    "S",        #106
]

In [11]:
#CNN Block
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, BatchNorm_ActiveFunc=True, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias = not bn_act, **kwargs)
        self.BatchNorm_ActiveFunc = nn.BatchNorm2d(out_channels) #?
        self.leaky = nn.LeakyReLU(0.1)
        self.use_BatchNorm_ActiveFunc = BatchNorm_ActiveFunc #?

    def forward(self, x):
        if(self.use_bn_act):
            return self.leaky(self.bn(self.conv(x)))
        else: 
            return self.conv(x)
    

# bn_act is if the block using Batch_Normalization function an an activation-function, true by default
# **kwargs just to enable the act to pass some more variable to the function

# if we use Batch_Normalization as an activation function, then the bias will not be used.

# Caution: the Negative Slope of LeakyReLU set to 0.1. that mean, when the input down below 0, then the output will be neg-Slope x Input
# ----------------------------------------------------------------------------
# if we use bn as an active function then, we caculate leakyReLu of Batch Normalization of output channel of conv

# else, just convlutional


In [ ]:
#Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, channels, use_residual=True, num_repeats=1):
        super().__init__()
        self.layer = nn.ModuleList()
        for repeat in range (num_repeats):
            sefl.layer += [
                nn.Sequential(
                    CNNBlock(channels, channels//2, kernel_size = 1),
                    CNNBlock(channels //2, channels, kernel_size = 3, padding = 1)
                )
            ]
        self.use_residual = use_residual
        self.num_repeats = num_repeats
        
    def forward(self, x):
        for layer in self.layers:
            if self.use_residual:
                x = x + layer(x)
            else:
                x = layer(x)

        return x


# CNN block with
# input channel is 3 as normal is RGB
# then the output is just 1? Why?

# if the kernel size is 3 and padding is 1 then output.shape is equal to input.shape

# this residual block is seted up as optional  if it need or not

# if we use residual block then:

# residual + input
# layer(x) +   x
# = output

# if not, then output = residual


In [ ]:
#Scale Prediction
class ScalePrediction(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.pred = nn.Sequential(
            CNNBlock(in_channels, 2 * in_channels, kernel_size=3, padding=1),
            CNNBlock( 2 * in_channels, 3 * (5 + num_classes), BatchNorm_ActiveFunc=False, kernel_size=1 )
            )
        self.num_classes = num_classes

    def forward(self, x):
        return (
            self.pred(x)
            .reshape(x.shape[0], 3, self.num_classes + 5, x.shape[2], x.shape[3])
            .permute(0, 1, 3, 4, 2)
        )
    
# nn.sequential is the order of the function inside it that will run in order as coded
# but the notion here is: the output of the previous is the input of the next one
    
    
# permute is just change the order of dimentional-layers 
# the output will depend on the order 1,2,3,...0

In [ ]:
#Yolo ver3
class YOLOOv3(nn.Module):
    pass

In [12]:
c=3
print (c//2)

1
